In [1]:
test_input = """#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#"""

In [2]:
input = open("inputs/13").read()

In [3]:
import numpy as np

def parse_board(input):
    return np.array(list(map(list, input.splitlines())))

def parse_input(input):
    return list(map(parse_board, input.split("\n\n")))

In [4]:
boards = parse_input(test_input)

In [5]:
board = boards[0]

In [6]:
num_cols = board.shape[1]
num_cols

9

In [7]:
# fold happens after (i-1)th column
# so, when i = 1, fold happens after 0th column
board[:, :8]

array([['#', '.', '#', '#', '.', '.', '#', '#'],
       ['.', '.', '#', '.', '#', '#', '.', '#'],
       ['#', '#', '.', '.', '.', '.', '.', '.'],
       ['#', '#', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '#', '.', '#', '#', '.', '#'],
       ['.', '.', '#', '#', '.', '.', '#', '#'],
       ['#', '.', '#', '.', '#', '#', '.', '#']], dtype='<U1')

In [8]:
board = boards[0]
(board[:, 1:5] == board[:, 5:][::, ::-1]).all()

True

In [9]:
board = boards[1]

(board[1:4, :] == board[4:, :][::-1, :]).all()

True

In [10]:
def flip_col_all_equal(a, b):
    assert a.shape == b.shape
    return np.all(a[:, ::-1] == b)

def check_all_col_folds(board):
    num_cols = board.shape[1]

    col_symms = []
    for i in range(1, num_cols):
        left_cols = i
        right_cols = num_cols - i

        left_side_shorter = left_cols < right_cols

        if left_side_shorter:
            eq = flip_col_all_equal(board[:, :i], board[:, i:(i + left_cols)])
        else:
            eq = flip_col_all_equal(board[:, (i - right_cols):i], board[:, i:])

        if eq:
            col_symms.append(i)
    
    return col_symms

def get_fold_summary(board):
    r1 = check_all_col_folds(board)
    r2 = check_all_col_folds(board.T)
    
    return r1[0] if r1 else 100*r2[0]

def get_all_fold_summaries(board):
    return check_all_col_folds(board) + [100*x for x in check_all_col_folds(board.T)]

In [11]:
get_fold_summary(boards[0]), get_fold_summary(boards[1])

(5, 400)

In [12]:
get_all_fold_summaries(boards[0]), get_all_fold_summaries(boards[1])

([5], [400])

In [13]:
boards = parse_input(input)

sum(get_fold_summary(board) for board in boards)

32035

In [14]:
sm = 0

for i, board in enumerate(boards):
    first_fold_summary = get_fold_summary(board)

    for index, c in np.ndenumerate(board):
        other_c = '.' if c == '#' else '#'

        board[index] = other_c
        rs = get_all_fold_summaries(board)
        board[index] = c

        r = next((r for r in rs if r != first_fold_summary), None)

        if r:
            sm += r
            break
    else:
        assert False

In [15]:
sm

24847